In [1]:
from sklearn.neural_network import MLPClassifier

In [2]:
from sklearn import preprocessing

In [3]:
from sklearn.model_selection import train_test_split 

In [4]:
X = [[0., 0.], [1., 1.]]

In [5]:
y = [0, 1]

In [6]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)

In [7]:
clf.fit(X,y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [8]:
clf.predict([[2., 2.], [-1., -2.]])

array([1, 0])

In [9]:
%run import_data

In [10]:
y = merged_data.grav

In [11]:
merged_data = merged_data.drop(['grav', 'num_veh','Num_Acc','locp','actp','etatp','catv','hrmn','mois','jour','catvopp','occutc'],axis = 1)

In [12]:
merged_data.columns

Index(['place', 'catu', 'sexe', 'secu', 'an_nais', 'senc', 'obs', 'obsm',
       'choc', 'manv', 'lum', 'agg', 'int', 'atm', 'col', 'mom', 'catv_gen',
       'catvopp_gen'],
      dtype='object')

In [13]:
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(merged_data)
merged_data = pd.DataFrame(np_scaled)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(merged_data, y, test_size=0.2, random_state=42)

In [15]:
clf = MLPClassifier(solver='adam', alpha=0.0001,hidden_layer_sizes=(128,64,32,32,64,2,32), random_state=42)

In [16]:
clf.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(128, 64, 32, 32, 64, 2, 32),
       learning_rate='constant', learning_rate_init=0.001, max_iter=200,
       momentum=0.9, nesterovs_momentum=True, power_t=0.5, random_state=42,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [17]:
resultat = clf.predict(X_test)

In [ ]:
resultat

In [ ]:
set(resultat)

In [ ]:
len(resultat)

In [ ]:
len(y_test)

In [ ]:
pourcentage = (resultat - y_test).replace(to_replace = [-4,-3,-2,-1,1,2,3,4], value = -1)


In [ ]:
print(1 + sum(pourcentage)/len(pourcentage))

In [ ]:
df = pd.DataFrame(index=merged_data.index)

In [ ]:
for colonne in merged_data.columns:
    essai_dummies = pd.get_dummies(data = merged_data[colonne], prefix = colonne)
    print(colonne)
    df = pd.concat([df, essai_dummies])

In [ ]:
essai_dummies.shape

In [ ]:
df.head
